In [1]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed,Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
import numpy as np

In [2]:
#Mount to drive where datasets are stored
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())

/workstation/home/jisna/PROTEINASSIGNMENT


In [ ]:
# Check if GPU available(for DGX/GPU server machine)
#import tensorflow as tf
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#In colab 
#Loading training data (9021 proteins)
x_train=np.load('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/datasetX_normal_500.npy')
y_train=np.load('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/datasetY_normal_500.npy')
print("x_train: " + str(x_train.shape))
print("y_train: " + str(y_train.shape))

#Loading test set (189 proteins)
x_test=np.load('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/bench_193x500x23_xyzjune21.npy')
y_test=np.load('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/bench_labels_193x500x3_xyzjune21.npy')
print("x_test: " + str(x_test.shape))
print("y_test: " + str(y_test.shape))

x_train: (103741500,)
y_train: (13531500,)
x_test: (2173500,)
y_test: (283500,)


In [4]:
#Reshaping the inputs for CNN-LSTM/BLSTM (2DCNN at first layer)
x_train = x_train.reshape(9021, 500, 23, 1)
y_train = y_train.reshape(9021, 500, 3)
#x_test and y_test represents benchmarked set throughout the code(coordinates and labels respectively)
x_test = x_test.reshape(189, 500, 23, 1)
y_test = y_test.reshape(189, 500, 3)

Testing by reloading the saved weights

In [13]:
#Load the saved model(2DCNN-BLSTM)
#!pip install keras
from keras.models import load_model
new_model = load_model('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/my_model_full9624Accuracy')
new_model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 500, 23, 32)       832       
_________________________________________________________________
batch_normalization_6 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 500, 23, 32)       25632     
_________________________________________________________________
batch_normalization_7 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 500, 23, 1)        801       
_________________________________________________________________
flatten_3 (Flatten)          (None, 11500)             0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 11500)            

In [14]:
resultbenchmarkCNNBLSTM = new_model.evaluate(x_test, y_test) # Accuracy on benchmark set(Test set I), for CNN-BLSTM model

6/6 [==============================] - 5s 692ms/step - loss: 0.1147 - accuracy: 0.9624


CNN-LSTM model

In [5]:
#Load the saved model(2DCNN-LSTM)
#!pip install keras
from keras.models import load_model
new_model = load_model('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/CNNLSTMTESTSETI')
new_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 500, 23, 32)       832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 500, 23, 32)       25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 500, 23, 1)        801       
_________________________________________________________________
flatten (Flatten)            (None, 11500)             0         
_________________________________________________________________
reshape (Reshape)            (None, 500, 23)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 500, 132)          82368     
_________________________________________________________________
time_distributed (TimeDistri (None, 500, 3)            3

In [6]:
resultbenchmarkCNNLSTM = new_model.evaluate(x_test, y_test) # Accuracy on benchmark set(Test set I), for CNN-LSTM model

6/6 [==============================] - 24s 719ms/step - loss: 0.1339 - accuracy: 0.9557


Executing CNN-LSTM model on constructed dataset and testing on Test set I.(Not by loading the saved model)

In [ ]:
#In system,
#Loading training set with 9021 samples
x_train=np.load('datasetX_normal_9021x500.npy')
y_train=np.load('datasetY_normal_9021x500.npy')
print("x_train: " + str(x_train.shape))
print("y_train: " + str(y_train.shape))

#Loading test set with 189 proteins
x_test=np.load('bench_193x500x23_xyzjune21.npy')
y_test=np.load('bench_labels_193x500x3_xyzjune21.npy')
print("x_test: " + str(x_test.shape))
print("y_test: " + str(y_test.shape))

x_train: (103741500,)
y_train: (13531500,)
x_test: (2173500,)
y_test: (283500,)


In [ ]:
#In system,
#Loading training set with 9021 samples
#x_train=np.load('datasetX_normal_22532x500.npy')
#y_train=np.load('datasetY_normal_22532x500.npy')
#print("x_train: " + str(x_train.shape))
#print("y_train: " + str(y_train.shape))

#Loading test set with 189 proteins
#x_test=np.load('bench_193x500x23_xyzjune21.npy')
#y_test=np.load('bench_labels_193x500x3_xyzjune21.npy')
#print("x_test: " + str(x_test.shape))
#print("y_test: " + str(y_test.shape))

In [ ]:
##CNN-LSTM model, model definition, summary and compilation (for benchmarking test set)    
model18 = Sequential([
    #layers.Masking(mask_value=0.0, input_shape=(500, 23, 1)),
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.Flatten(),
    #Dropout(0.2),#3 to 2
    layers.Reshape([500, 23]),
    layers.LSTM(132, return_sequences=True),
    layers.TimeDistributed(Dense(3, activation = "softmax"))
])

model18.summary()
#opt = keras.optimizers.Adam(learning_rate=0.00001)

model18.compile(
    optimizer='adam',
   # loss='kullback_leibler_divergence',
     loss='categorical_crossentropy',
    metrics=['accuracy']
)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 500, 23, 32)       832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 500, 23, 32)       25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 500, 23, 1)        801       
_________________________________________________________________
flatten (Flatten)            (None, 11500)             0         
_________________________________________________________________
reshape (Reshape)            (None, 500, 23)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 500, 132)          82368     
_________________________________________________________________
time_distributed (TimeDistri (None, 500, 3)            3

In [ ]:
!watch -n 2 nvidia-smi

>---------------------------------------------------------------------------+8ue Jun 15 06:01:37 2021

In [ ]:
#Model Training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='3'
# using GPU 
os.environ["CUDA_VISIBLE_DEVICES"]='1'
with tf.device("/gpu:1"):
    fit1 = model18.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
    #callbacks=[callback]
)
#100 32 .1 96.34 95.06 95.10


Epoch 1/50
226/226 [==============================] - 13s 57ms/step - loss: 0.4092 - accuracy: 0.3661 - val_loss: 0.3320 - val_accuracy: 0.8500
Epoch 2/50
226/226 [==============================] - 12s 51ms/step - loss: 0.2298 - accuracy: 0.8971 - val_loss: 0.1975 - val_accuracy: 0.9115
Epoch 3/50
226/226 [==============================] - 11s 50ms/step - loss: 0.1737 - accuracy: 0.9215 - val_loss: 0.1586 - val_accuracy: 0.9277
Epoch 4/50
226/226 [==============================] - 11s 50ms/step - loss: 0.1674 - accuracy: 0.9245 - val_loss: 0.1539 - val_accuracy: 0.9301
Epoch 5/50
226/226 [==============================] - 11s 51ms/step - loss: 0.1493 - accuracy: 0.9319 - val_loss: 0.1510 - val_accuracy: 0.9316
Epoch 6/50
226/226 [==============================] - 12s 51ms/step - loss: 0.1476 - accuracy: 0.9326 - val_loss: 0.1482 - val_accuracy: 0.9322
Epoch 7/50
226/226 [==============================] - 11s 51ms/step - loss: 0.1426 - accuracy: 0.9350 - val_loss: 0.1407 - val_accuracy:

In [ ]:
#Testing using benchmarked dataset(Test set I), accuracy varies over a small percentage depending on the random initilization of weights
results1 = model18.evaluate(x_test, y_test)

6/6 [==============================] - 0s 28ms/step - loss: 0.1220 - accuracy: 0.9516


Saving the above CNN-LSTM model-model18

In [ ]:
#full model
model18.save('my_model_fullCNNLSTM9519Accuracy')
#weights only
model18.save_weights('my_model_weightsCNNLSTM9519Accuracy.h5')

To view the loss and accuracy plots

In [ ]:
!pip3 install matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit1.history['accuracy'])
plt.plot(fit1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/accuracy96Accuracy.eps')

# loss graph
plt.plot(fit1.history['loss'])
plt.plot(fit1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('loss9550Acculoss.eps')
#plt.savefig('loss9550Acculoss.pdf')

Training and testing(Test set II) on the newly constructed dataset. Here testing is done on 2000 proteins randomly sampled from new dataset.

In [ ]:
# Then Split dataset randomly for training and testing
# 2000 for testset and remaining for training
import numpy as np
s = np.random.choice(range(x_train.shape[0]), 2000, replace=False)

s1 = list(set(range(x_train.shape[0])) - set(s))

x_train_our = x_train[s1, :, :]
y_train_our = y_train[s1, :, :]

#x_val_s = x_train[s2, :, :] #if separate validation set is needed
#y_val_s = y_train[s2, :, :]

x_test_our = x_train[s, :, :]
y_test_our = y_train[s, :, :]

print(x_train_our.shape)
print(y_train_our.shape)
#print(x_val_s.shape)
#print(y_val_s.shape)
print(x_test_our.shape)
print(y_test_our.shape)

del s, s1#,x_train, y_train

(7021, 500, 23, 1)
(7021, 500, 3)
(2000, 500, 23, 1)
(2000, 500, 3)


In [ ]:
#Reshaping the inputs for CNN-LSTM/BLSTM (2DCNN at first layer)
x_train_our = x_train_our.reshape(7021, 500, 23, 1)
y_train_our = y_train_our.reshape(7021, 500, 3)

x_test_our = x_test_our.reshape(2000, 500, 23, 1)
y_test_our = y_test_our.reshape(2000, 500, 3)

In [ ]:
##CNN-LSTM model, model definition, summary and compilation 
model18LSTMNEWTEST = Sequential([
    #layers.Masking(mask_value=0.0, input_shape=(500, 23, 1)),
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.Flatten(),
    #Dropout(0.2),#3 to 2
    layers.Reshape([500, 23]),
    layers.LSTM(128, return_sequences=True),
    layers.TimeDistributed(Dense(3, activation = "softmax"))
])

model18LSTMNEWTEST.summary()
#opt = keras.optimizers.Adam(learning_rate=0.00001)

model18LSTMNEWTEST.compile(
    optimizer='adam',
   # loss='kullback_leibler_divergence',
     loss='categorical_crossentropy',
    metrics=['accuracy']
)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 500, 23, 32)       832       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 500, 23, 32)       25632     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 500, 23, 32)       25632     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 500, 23, 1)        801       
_________________________________________________________________
flatten_3 (Flatten)          (None, 11500)             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 500, 23)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 500, 128)         

In [ ]:
#Model Training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='3'
# using GPU 
os.environ["CUDA_VISIBLE_DEVICES"]='1'
with tf.device("/gpu:1"):
    fit18LSTM = model18LSTMNEWTEST.fit(
    x_train_our, y_train_our,
    epochs=30,
    batch_size=32,
    validation_split=0.2
    #callbacks=[callback]
)

Epoch 1/30
176/176 [==============================] - 10s 60ms/step - loss: 0.4640 - accuracy: 0.2065 - val_loss: 0.4381 - val_accuracy: 0.2206
Epoch 2/30
176/176 [==============================] - 10s 58ms/step - loss: 0.3222 - accuracy: 0.5703 - val_loss: 0.2146 - val_accuracy: 0.9029
Epoch 3/30
176/176 [==============================] - 10s 57ms/step - loss: 0.1964 - accuracy: 0.9109 - val_loss: 0.1768 - val_accuracy: 0.9203
Epoch 4/30
176/176 [==============================] - 10s 56ms/step - loss: 0.1630 - accuracy: 0.9260 - val_loss: 0.1673 - val_accuracy: 0.9236
Epoch 5/30
176/176 [==============================] - 10s 57ms/step - loss: 0.1535 - accuracy: 0.9298 - val_loss: 0.1497 - val_accuracy: 0.9310
Epoch 6/30
176/176 [==============================] - 10s 56ms/step - loss: 0.1475 - accuracy: 0.9319 - val_loss: 0.1464 - val_accuracy: 0.9324
Epoch 7/30
176/176 [==============================] - 10s 57ms/step - loss: 0.1427 - accuracy: 0.9340 - val_loss: 0.1407 - val_accuracy:

In [ ]:
#Testing using new testset, Accuracy varies on a small percentage depending on the random initilization of weights
results1 = model18LSTMNEWTEST.evaluate(x_test_our, y_test_our)

63/63 [==============================] - 1s 20ms/step - loss: 0.1136 - accuracy: 0.9487


To save th model and weights

In [ ]:
#full model
model18LSTMNEWTEST.save('my_model_full18CNNLSTM9518NEWTEST95Accuracy')
#weights only
model18LSTMNEWTEST.save_weights('my_model_weights18CNNLSTMNEWTEST95Accuracy.h5')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model_full18CNNLSTM9518NEWTEST95Accuracy/assets


In [9]:
#!pip install keras
from keras.models import load_model
new_modelCNNLSTM = load_model('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/CNNLSTM95model')

In [10]:
new_modelCNNLSTM.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 500, 23, 32)       832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 500, 23, 32)       25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 500, 23, 1)        801       
_________________________________________________________________
flatten (Flatten)            (None, 11500)             0         
_________________________________________________________________
reshape (Reshape)            (None, 500, 23)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 500, 132)          82368     
_________________________________________________________________
time_distributed (TimeDistri (None, 500, 3)            3

In [ ]:
#Testing using new testset
results_setII = new_modelCNNLSTM.evaluate(x_test_our, y_test_our)

63/63 [==============================] - 1s 19ms/step - loss: 0.0898 - accuracy: 0.9611


In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit1.history['accuracy'])
plt.plot(fit1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('accuracyCNNLSTM9519Accuracy.eps')

# loss graph
plt.plot(fit1.history['loss'])
plt.plot(fit1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('loss9550Acculoss.eps')
plt.savefig('lossCNNLSTM9519loss.pdf')

CNN-BLSTM model execution on Test set I(first model)

In [ ]:
#CNN-BLSTM model, model definition, summary and compilation
hid_units=128
model18BLSTM1 = Sequential([
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
     layers.BatchNormalization(),
     layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
     #layers.Flatten(),
     layers.Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same'),#or 1x1
     layers.Flatten(),
     layers.Dropout(0.2),
     layers.Reshape([500, 23]),
     layers.Bidirectional(LSTM(hid_units,  activation = "tanh", return_sequences=True), merge_mode="concat",input_shape=(500,23)),
     #layers.LSTM(128, return_sequences=True),
     Dense(512, activation='relu'),
    layers.Dropout(0.2),
     Dense(256, activation='relu'),
     layers.Dropout(0.2),
    Dense(128, activation='relu'),
    layers.Dropout(0.3),
     #Dense(3, activation='softmax')
     layers.TimeDistributed(Dense(3, activation = "softmax"))
])

model18BLSTM1.summary()

model18BLSTM1.compile(
    optimizer='adam',
    #loss='kullback_leibler_divergence',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 500, 23, 32)       832       
_________________________________________________________________
batch_normalization (BatchNo (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 500, 23, 32)       25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 500, 23, 1)        801       
_________________________________________________________________
flatten_1 (Flatten)          (None, 11500)             0         
_________________________________________________________________
dropout (Dropout)            (None, 11500)            

In [ ]:
#Model Training

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='3'
# using GPU 
os.environ["CUDA_VISIBLE_DEVICES"]='1'
with tf.device("/gpu:1"):
    fit11 = model18BLSTM1.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
    #callbacks=[callback]
)
    

Epoch 1/50
226/226 [==============================] - 22s 97ms/step - loss: 0.0768 - accuracy: 0.9593 - val_loss: 0.0892 - val_accuracy: 0.9560
Epoch 2/50
226/226 [==============================] - 22s 97ms/step - loss: 0.0771 - accuracy: 0.9606 - val_loss: 0.0896 - val_accuracy: 0.9334
Epoch 3/50
226/226 [==============================] - 22s 96ms/step - loss: 0.0771 - accuracy: 0.9588 - val_loss: 0.0886 - val_accuracy: 0.9529
Epoch 4/50
226/226 [==============================] - 22s 97ms/step - loss: 0.0760 - accuracy: 0.9609 - val_loss: 0.0879 - val_accuracy: 0.9549
Epoch 5/50
226/226 [==============================] - 22s 97ms/step - loss: 0.0760 - accuracy: 0.9618 - val_loss: 0.0888 - val_accuracy: 0.9561
Epoch 6/50
226/226 [==============================] - 22s 96ms/step - loss: 0.0755 - accuracy: 0.9610 - val_loss: 0.0898 - val_accuracy: 0.9563
Epoch 7/50
226/226 [==============================] - 22s 97ms/step - loss: 0.0760 - accuracy: 0.9606 - val_loss: 0.0887 - val_accuracy:

In [ ]:
#Testing using benchmarked dataset
results11 = model18BLSTM1.evaluate(x_test, y_test)

6/6 [==============================] - 0s 31ms/step - loss: 0.1121 - accuracy: 0.9537


CNN-BLSTM model execution on Test set I(second model-minor modifications to first one also gives comparable accuracies)

In [ ]:
#In system,
#Loading training set with 9021 samples
x_train=np.load('datasetX_normal_9021x500.npy')
y_train=np.load('datasetY_normal_9021x500.npy')
print("x_train: " + str(x_train.shape))
print("y_train: " + str(y_train.shape))

#Loading test set with 189 proteins
x_test=np.load('bench_193x500x23_xyzjune21.npy')
y_test=np.load('bench_labels_193x500x3_xyzjune21.npy')
print("x_test: " + str(x_test.shape))
print("y_test: " + str(y_test.shape))

In [ ]:
#Reshaping the inputs for CNN-LSTM/BLSTM (2DCNN at first layer)
x_train = x_train.reshape(9021, 500, 23, 1)
y_train = y_train.reshape(9021, 500, 3)
#x_test and y_test represents benchmarked set throughout the code(coordinates and labels respectively)
x_test = x_test.reshape(189, 500, 23, 1)
y_test = y_test.reshape(189, 500, 3)

In [ ]:
#CNN-BLSTM model, model definition, summary and compilation
hid_units=128
model18BLSTM2 = Sequential([
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
     layers.BatchNormalization(),
     layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
     #layers.Flatten(),
     layers.Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same'),#or 1x1
     layers.Flatten(),
    layers.Dropout(0.3),
     layers.Reshape([500, 23]),
     layers.Bidirectional(LSTM(hid_units,  activation = "tanh", return_sequences=True), merge_mode="concat",input_shape=(500,23)),
    # layers.LSTM(132, return_sequences=True),
     Dense(512, activation='relu'),
    layers.Dropout(0.3),
     Dense(256, activation='relu'),
   layers.Dropout(0.3),
    Dense(128, activation='relu'),
    layers.Dropout(0.3),
     #Dense(3, activation='softmax')
     layers.TimeDistributed(Dense(3, activation = "softmax"))
])

model18BLSTM2.summary()

model18BLSTM2.compile(
    optimizer='adam',
    #loss='kullback_leibler_divergence',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#CCE  100 32 .1 BLSTM+LSTM in one-cannot use
#

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 500, 23, 32)       832       
_________________________________________________________________
batch_normalization_2 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 500, 23, 32)       25632     
_________________________________________________________________
batch_normalization_3 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 500, 23, 1)        801       
_________________________________________________________________
flatten_2 (Flatten)          (None, 11500)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 11500)            

In [ ]:
#Model Training

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='3'
# using GPU 
os.environ["CUDA_VISIBLE_DEVICES"]='1'
with tf.device("/gpu:1"):
    fit12 = model18BLSTM2.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
    #callbacks=[callback]
)
    #508/508 [==============================] - 28s 56ms/step - loss: 0.1004 - accuracy: 0.9536 - val_loss: 0.1076 - val_accuracy: 0.9516 and 95.05

Epoch 1/50
226/226 [==============================] - 23s 101ms/step - loss: 0.1021 - accuracy: 0.9527 - val_loss: 0.0968 - val_accuracy: 0.9544
Epoch 2/50
226/226 [==============================] - 22s 99ms/step - loss: 0.0969 - accuracy: 0.9550 - val_loss: 0.0960 - val_accuracy: 0.9558
Epoch 3/50
226/226 [==============================] - 22s 100ms/step - loss: 0.0962 - accuracy: 0.9552 - val_loss: 0.0982 - val_accuracy: 0.9549
Epoch 4/50
226/226 [==============================] - 23s 100ms/step - loss: 0.0969 - accuracy: 0.9551 - val_loss: 0.1085 - val_accuracy: 0.9483
Epoch 5/50
226/226 [==============================] - 22s 99ms/step - loss: 0.0967 - accuracy: 0.9543 - val_loss: 0.0973 - val_accuracy: 0.9553
Epoch 6/50
226/226 [==============================] - 23s 100ms/step - loss: 0.0946 - accuracy: 0.9556 - val_loss: 0.0966 - val_accuracy: 0.9547
Epoch 7/50
226/226 [==============================] - 23s 100ms/step - loss: 0.0951 - accuracy: 0.9560 - val_loss: 0.0989 - val_accu

In [ ]:
#Testing using benchmarked dataset
results18BLSTM2 = model18BLSTM2.evaluate(x_test, y_test)

6/6 [==============================] - 0s 28ms/step - loss: 0.1125 - accuracy: 0.9571


In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit12.history['accuracy'])
plt.plot(fit12.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/accuracy88Accuracy.eps')

# loss graph
plt.plot(fit12.history['loss'])
plt.plot(fit12.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/loss88Acculoss.eps')

On test set II(x_test_our and y_test_our)

In [ ]:
#CNN-BLSTM model, model definition, summary and compilation (for benchmarking test set)
hid_units=128
model18BLSTM3 = Sequential([
    layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
     layers.BatchNormalization(),
     layers.Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same',input_shape=(500,23,1)),
     layers.BatchNormalization(),
     #layers.Flatten(),
     layers.Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same'),#or 1x1
     layers.Flatten(),
     #layers.Dropout(0.2),
     layers.Reshape([500, 23]),
     layers.Bidirectional(LSTM(hid_units,  activation = "tanh", return_sequences=True), merge_mode="concat",input_shape=(500,23)),
     #layers.LSTM(128, return_sequences=True),
     #Dense(512, activation='relu'),
     #layers.Dropout(0.2),
     #Dense(256, activation='relu'),
     #layers.Dropout(0.2),
     Dense(128, activation='relu'),
     layers.Dropout(0.2),
     #Dense(3, activation='softmax')
     layers.TimeDistributed(Dense(3, activation = "softmax"))
])

model18BLSTM3.summary()

model18BLSTM3.compile(
    optimizer='adam',
   # loss='kullback_leibler_divergence',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#32-5,32-5,1-5,.5,BLSTM-64,512-.5,256-.5,128-.5 for 100,32,0.1 =

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 500, 23, 32)       832       
_________________________________________________________________
batch_normalization_8 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 500, 23, 32)       25632     
_________________________________________________________________
batch_normalization_9 (Batch (None, 500, 23, 32)       128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 500, 23, 1)        801       
_________________________________________________________________
flatten_5 (Flatten)          (None, 11500)             0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 500, 23)          

In [ ]:
#Model Training

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='3'
# using GPU 
os.environ["CUDA_VISIBLE_DEVICES"]='1'
with tf.device("/gpu:1"):
    fit2 = model18BLSTM3.fit(
    x_train_our, y_train_our,
    epochs=50,
    batch_size=32,
    validation_split=0.2
    #callbacks=[callback]
)

Epoch 1/50
176/176 [==============================] - 18s 102ms/step - loss: 0.4084 - accuracy: 0.5593 - val_loss: 0.4239 - val_accuracy: 0.7875
Epoch 2/50
176/176 [==============================] - 17s 96ms/step - loss: 0.2361 - accuracy: 0.8924 - val_loss: 0.3438 - val_accuracy: 0.8470
Epoch 3/50
176/176 [==============================] - 17s 95ms/step - loss: 0.1773 - accuracy: 0.9195 - val_loss: 0.1816 - val_accuracy: 0.9170
Epoch 4/50
176/176 [==============================] - 17s 96ms/step - loss: 0.1604 - accuracy: 0.9275 - val_loss: 0.1497 - val_accuracy: 0.9318
Epoch 5/50
176/176 [==============================] - 17s 97ms/step - loss: 0.1472 - accuracy: 0.9333 - val_loss: 0.1425 - val_accuracy: 0.9352
Epoch 6/50
176/176 [==============================] - 17s 96ms/step - loss: 0.1535 - accuracy: 0.9315 - val_loss: 0.1358 - val_accuracy: 0.9388
Epoch 7/50
176/176 [==============================] - 17s 95ms/step - loss: 0.1372 - accuracy: 0.9381 - val_loss: 0.1429 - val_accuracy

In [ ]:
#Testing using benchmarked dataset
results18BLSTM3 = model18BLSTM3.evaluate(x_test_our, y_test_our)

63/63 [==============================] - 2s 35ms/step - loss: 0.0956 - accuracy: 0.9551


In [ ]:
#full model
model18BLSTM3.save('my_model_full18CNNBLSTM9518NEWTEST955Accuracy')
#weights only
model18BLSTM3.save_weights('my_model_weights18CNNBLSTMNEWTEST955Accuracy.h5')

INFO:tensorflow:Assets written to: my_model_full18CNNBLSTM9518NEWTEST955Accuracy/assets


In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit2.history['accuracy'])
plt.plot(fit2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/accuracy88Accuracy.eps')

# loss graph
plt.plot(fit2.history['loss'])
plt.plot(fit2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/loss88Acculoss.eps')

In [ ]:
#Do splitting earlier and load it directly
#x_train_our=np.load('/home/jisna/Downloads/train_7221x500x23_xyz.npy')
#y_train_our=np.load('/home/jisna/Downloads/train_labels_7221x500x3_xyz.npy')
#x_train_our = x_train_our.reshape(8121, 500, 23, 1) 
#y_train_our = y_train_our.reshape(8121, 500, 3)
#print("x_train_our: " + str(x_train_our.shape))
#print("y_train_our: " + str(y_train_our.shape))

#x_val=np.load('/home/jisna/Downloads/val_900x500x23_xyz.npy')
#y_val=np.load('/home/jisna/Downloads/val_labels_900x500x3_xyz.npy')
#x_val = x_val.reshape(900, 500, 23)
#y_val = y_val.reshape(900, 500, 3)
#print("x_val: " + str(x_val.shape))
#print("y_val: " + str(y_val.shape))

#x_test_our=np.load('/home/jisna/Downloads/test_900x500x23_xyz.npy')
#y_test_our=np.load('/home/jisna/Downloads/test_labels_900x500x3_xyz.npy')
#x_test_our = x_test_our.reshape(900, 500, 23, 1)
#y_test_our = y_test_our.reshape(900, 500, 3)
#print("x_test_our: " + str(x_test_our.shape))
#print("y_test_our: " + str(y_test_our.shape))

In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit3.history['accuracy'])
plt.plot(fit3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/accuracy88Accuracy.eps')

# loss graph
plt.plot(fit3.history['loss'])
plt.plot(fit3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/loss88Acculoss.eps')

In [ ]:
#Test on individual proteins from benchmark set

In [ ]:
x_test1=x_test[169].reshape(1,500,23,1)
x_test1.shape
y_test1=y_test[169]
y_test1.shape

In [23]:

ti = 2 #third protein in test set

preds = new_model.predict(x_test[ti].reshape(1, 500, 23, 1))

label_index = np.argmax(preds, axis=2)

labels = [1, 0, 0, 0, 1, 0, 0, 0, 1]

labels = np.array(labels)
labels = labels.reshape(3, 3)

#print(labels)

t = 0
x = 0
for i, j in zip(label_index[0], y_test[ti]):
  if (str('[0 0 0]') == str(j)):
    break
  t += 1
  if (str(labels[i]) != str(j)):
    x += 1
t=t-1
x=x-1
print("Mismatch in classes: " + str(x) + "\nLength of sequence: " + str(t))
print("Length of sequence: " + str(t))
print("Accuracy: " + str(((t-x)/t)*100) + "%")
for k in range(0,t):
  if (label_index[0][k])==0:
    print("H",end=' ')
  if (label_index[0][k])==1:
    print("E",end=' ')
  if (label_index[0][k])==2:
    print("C",end=' ')
  #print(label_index[0][k])

Mismatch in classes: 3
Length of sequence: 109
Length of sequence: 109
Accuracy: 97.24770642201835%
C C C C H H H H H H H H H H H H H C C C C H H H C C C C C C C C C C C H H H H C C C C C C H H H H H H H H H C C C C C C H H H H H H H H H H H H H H H H H H C C C C C H H H H H H H H H H H H H H H H H H C C C C C H H C 

In [21]:

ti =5 # sixth protein in test set

preds = new_model.predict(x_test[ti].reshape(1, 500, 23, 1))

label_index = np.argmax(preds, axis=2)

labels = [1, 0, 0, 0, 1, 0, 0, 0, 1]

labels = np.array(labels)
labels = labels.reshape(3, 3)

#print(labels)

t = 0
x = 0
for i, j in zip(label_index[0], y_test[ti]):
  if (str('[0 0 0]') == str(j)):
    break
  t += 1
  if (str(labels[i]) != str(j)):
    x += 1
t=t-1
x=x-1
print("Mismatch in classes: " + str(x) + "\nLength of sequence: " + str(t))
print("Length of sequence: " + str(t))
print("Accuracy: " + str(((t-x)/t)*100) + "%")
for k in range(0,t):
  if (label_index[0][k])==0:
    print("H",end=' ')
  if (label_index[0][k])==1:
    print("E",end=' ')
  if (label_index[0][k])==2:
    print("C",end=' ')
  #print(label_index[0][k])

Mismatch in classes: 2
Length of sequence: 120
Length of sequence: 120
Accuracy: 98.33333333333333%
C H H H H H C H H H H H H H H H H H H H H H H H H C C C C H H H H C C C C H H H H H H H H H H H H H H H C C C C C C C C C C C H H H H H C C C C C C C C H H H H H H H H H H H H H H H H H C C C C H H H H H H H H H H H H H C H H H H C C C C C 

In [ ]:
#check this also
label_index = np.argmax(result, axis=2)

labels = [1, 0, 0, 0, 1, 0, 0, 0, 1]

labels = np.array(labels)
labels = labels.reshape(3, 3)

#print(labels)

t = 0
x = 0
for i, j in zip(label_index[0], y_test1):
  if (str('[0 0 0]') == str(j)):
    break
  t += 1
  if (str(labels[i]) != str(j)):
    x += 1
#t=t-1
x=x-1 # first residue is skipped
print("Mismatch in classes: " + str(x) + "\nLength of sequence: " + str(t))
print("Length of sequence: " + str(t))
print("Accuracy: " + str(((t-x)/t)*100) + "%")
print("C",end=' ')
for k in range(0,t):
  if (label_index[0][k])==0:
    print("H",end=' ')
  if (label_index[0][k])==1:
    print("E",end=' ')
  if (label_index[0][k])==2:
    print("C",end=' ')
  #print(label_index[0][k])


In [ ]:
x_test1=x_test[169].reshape(1,500,23,1)
x_test1.shape
y_test1=y_test[169]
y_test1.shape

In [ ]:
#----------Plot the graph----------

In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.plot(fit6.history['accuracy'])
plt.plot(fit6.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/accuracy88Accuracy.eps')

# loss graph
plt.plot(fit6.history['loss'])
plt.plot(fit6.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#plt.savefig('/content/drive/MyDrive/CNN_for_STR_ASSIGNMENT_DATA_and_CODES/loss88Acculoss.eps')